<a href="https://colab.research.google.com/github/zliobaite/TBIteaching2024/blob/main/Practical1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Practical 1: NOW database**

In this exercise you will explore the NOW database of fossil mammals.

Task: compare the number of mammalian fossil localities in Kenya, Tanzania and Ethiopia over the time period from 34 million years ago to today. Does the number of localities increase/decrease/stay similar over time? How does the number of localities compare accross countries through time?


**Step 1:** explore the database

Go to the NOW database https://nowdatabase.luomus.fi/ . Go to "Locality" tab. Enter "Kenya" in the "Country or Continent" search box. How many localities are found? Try to open one or a few, see what is in there.

**Step 2:** download a csv with a list of localities from Kenya, Tanzania and Ethiopia

Enter "Kenya,Tanzania,Ethiopia" in the "Country or Continent" search box. How many localities are found?

Click "Export". In the pop up box tick "Export to file" and click "Selected localities/species". Rename the file with a shorter name for convenience, for example "now_KeTaEt.csv". Open the csv, explore it.

**Step 3:** read the data from the csv, compute middle age point for each locality

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
import io

df = pd.read_csv(io.StringIO(uploaded['now_KeTaEt.csv'].decode('utf-8')),delimiter='\t')
df